In [1]:
from RzLinear import RzLinearFunction
import torch
from RzLinear import RzLinear 
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import rz_linear

In [2]:
TILED = True
input_dim = 100
output_dim = 100
weight_size = 1000
chunk_size = 2
r = np.random.RandomState(1024)
x = r.randint(0, 2038074743, (50,))
x = x + 1*(x%2==0);
random_numbers = torch.from_numpy(np.concatenate([np.array([2038074743]), x])).long().cuda(0) # set of 50 random numbers to use

In [3]:
idx =rz_linear.get_idx(random_numbers, input_dim, output_dim, chunk_size, weight_size, TILED)

In [4]:
#hashed_weight = nn.Parameter(torch.from_numpy(np.arange(weight_size).astype(np.float32))).to("cuda:0")
hashed_weight = nn.Parameter(torch.rand((weight_size))).to("cuda:0")

In [5]:
weight = nn.Parameter(torch.clone(hashed_weight[idx])).cuda(0)

In [6]:
def forward_actual(inp, wt):
    return torch.mm(inp, wt)

def forward_rz(inp, hwt):
    return RzLinearFunction.forwardproxy(hwt, inp ,random_numbers, input_dim, output_dim, chunk_size, TILED)

input_v = nn.Parameter(torch.rand((5,input_dim)), requires_grad=True).cuda(0)
A = forward_actual(input_v, weight)
B = forward_rz(input_v, hashed_weight)
torch.max(torch.abs(A - B))

tensor(1.3351e-05, device='cuda:0', grad_fn=<MaxBackward1>)

In [7]:
def forward_actual(inp, wt):
    X = torch.matmul(inp, wt)
    return torch.sum(X)

def forward_rz(inp, hwt):
    X= RzLinearFunction.forwardproxy(hwt, inp ,random_numbers, input_dim, output_dim, chunk_size, TILED)
    return torch.sum(X)

def backward_actual(inp, wt):
    A = forward_actual(inp, wt)
    A.backward()
    return wt.grad


def backward_rz(inp, hwt):
    B = forward_rz(inp, hwt)
    grad = torch.ones((inp.shape[0], output_dim)).cuda(0)
    in_grad, wt_grad = RzLinearFunction.backwardproxy(grad, hwt, inp, random_numbers, input_dim, output_dim, chunk_size, TILED)
    return in_grad, wt_grad

In [8]:
input_v = nn.Parameter(torch.rand((5,input_dim)), requires_grad=True).cuda(0)
in_grad, wt_grad = backward_rz(input_v, hashed_weight)
wt_grad_2 = backward_actual(input_v, weight)

In [9]:
wt_grad_actual_p = torch.zeros_like(hashed_weight).cuda(0)
wt_grad_actual_p.scatter_add_(0, idx.reshape(-1), wt_grad_2.reshape(-1))
print(torch.max(torch.abs(wt_grad_actual_p - wt_grad)))

tensor(1.1444e-05, device='cuda:0')
